In [ ]:

# Save figures in specific place

knitr::opts_chunk$set(autodep        = TRUE,
                      cache          = FALSE,
                      cache.comments = TRUE,

                      # Save figures as pdf ?
                      dev = c("pdf"),

                      # Include code?
                      echo           = TRUE,

                      error          = FALSE,
                      fig.align      = "center",

                      # Path where figures are going to be store pdf single
                      # figures
                      fig.path       = paste0("./figures", "/"),
                      fig.width      = 11,
                      fig.height     = 7,
                      message        = FALSE)

# set to two decimal
options(scipen = 1, digits = 4)

In [ ]:
library(dplyr)
library(performance)
library(tibble)
library(here)
library(emmeans)
library(car)
library(modelr)
library(knitr)
library(ggpubr)
library(janitor)
library(factoextra)
library(dichromat)

In [ ]:
# Set knit directory
setwd(here::here())
knitr::opts_knit$set(root.dir = setwd(here::here()))
getwd()

# Load data

In [ ]:
# Load all joined dataset
source("./scripts/code_join_data_full_dataset.R")

__Q3: Are traits coordinated along the "fast to slow" continuum and to what extent does this correlated with N-fixing status?__

## Trait PCA

In [ ]:
# Select variables that are going to be used in the model
data_for_models <-
    data_for_models %>%

        clean_names() %>%
        # Select columns needed for regression
        dplyr::select(spcode, treatment, nfixer, init_height,

                  # Performance measures
                  total_biomass, rgr, root_shoot_ratio,

                  # physiology traits
                  amax, gs, wue, pnue, sla,

                  # Leaf traits
                  narea_g_m2) %>%
        clean_names()

In [ ]:
data_traits_scaled <-
    data_for_models %>%
        dplyr::select(-c(total_biomass, rgr, root_shoot_ratio, init_height)) %>%

        mutate(across(where(is.numeric), scale))


In [ ]:
pca_traits <- princomp(data_traits_scaled[, c(4:ncol(data_traits_scaled))])

In [ ]:
eigs <- pca_traits$sdev^2/sum(pca_traits$sdev^2)

In [ ]:
fviz_screeplot(pca_traits, addlabels = TRUE, ylim = c(0, 50))

In [ ]:
fviz_pca_biplot(pca_traits, axes = c(1, 2),

                invisible="quali",
                pointsize = 2.5,
                # Add group
                habillage= data_for_models$nfixer,

                # Set labels
                label ="var",

                # Set arrow color
                col.var = "black",
                geom=c("point", "arrows"),
                # Repel overlapping labels
                repel = TRUE) +

  theme_minimal() + xlim(NA, 7.5) +  labs(title = "")+
  coord_fixed() + theme(text = element_text(size = 30))+
  xlab(paste0("PC1 [", round(100*eigs[1], 1), "%]")) +
  ylab(paste0("PC2 [", round(100*eigs[2], 1), "%]")) +

  scale_shape_manual(values = c(16, 17)) +
  scale_color_manual(values = c("#D95F02","#1B9E77" )) +
  scale_fill_manual(values = c("#D95F02","#1B9E77")) +
  theme(legend.position = "bottom")

In [ ]:
fviz_pca_biplot(pca_traits, axes = c(1, 3),

                invisible="quali",
                pointsize = 2.5,
                # Add group
                habillage= data_for_models$nfixer,

                # Set labels
                label ="var",

                # Set arrow color
                col.var = "black",
                geom=c("point", "arrows"),

                # Repel overlapping labels
                repel = TRUE) +

  theme_minimal() + xlim(NA, 7.5) +  labs(title = "")+
  coord_fixed() + theme(text = element_text(size = 30))+
  xlab(paste0("PC1 [", round(100*eigs[1], 1), "%]")) +
  ylab(paste0("PC3 [", round(100*eigs[3], 1), "%]")) +

  scale_shape_manual(values = c(16, 17)) +
  scale_color_manual(values = c("#D95F02","#1B9E77" )) +
  scale_fill_manual(values = c("#D95F02","#1B9E77")) +
  theme(legend.position = "bottom")

In [ ]:
fviz_pca_biplot(pca_traits, axes = c(2,3),

                invisible="quali",
                pointsize = 2.5,
                # Add group
                habillage= data_for_models$nfixer,

                # Set labels
                label ="var",

                # Set arrow color
                col.var = "black",
                geom=c("point", "arrows"),

                # Repel overlapping labels
                repel = TRUE) +

  theme_minimal() + xlim(NA, 7.5) +  labs(title = "")+
  coord_fixed() + theme(text = element_text(size = 30))+
  xlab(paste0("PC2 [", round(100*eigs[2], 1), "%]")) +
  ylab(paste0("PC3 [", round(100*eigs[3], 1), "%]")) +

  scale_shape_manual(values = c(16, 17)) +
  scale_color_manual(values = c("#D95F02","#1B9E77" )) +
  scale_fill_manual(values = c("#D95F02","#1B9E77")) +
  theme(legend.position = "bottom")

In [ ]:
# Extract loadings
pca_traits$loadings